In [6]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import math

In [3]:
def jprint(obj):
    """Create a formatted string of the Python JSON object"""
    
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

def moving_average(series, window, lag=0):
    """Calculate moving average of a series"""
    
    series = series.rolling(window=window).mean().tolist()
    series = [np.nan]*lag + series[0:len(series) - lag]
    return series

def get_yfinance_data(ticker):
    """Returns data from yahoo finance"""
    
    df = yf.Ticker(ticker) 
    df_hist = df.history(period="max")[['Open', 'High', 'Low', 'Close']]
    
    return df_hist

def makedir_p(path):
    """Returns a directory"""

    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

def model_metrics(y_pred, y_test, thresh):
    """Prints model metrics"""
    
    # common metrics
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    # return metric
    correct_return = 0
    for i in range(len(y_pred)):
        if (y_pred[i] > 0 and y_test[i]) > 0 or (y_pred[i] < 0 and y_test[i] < 0):
            if abs(y_pred[i] - y_test[i]) > thresh:
                correct_return += 1
    
    print('Testing set size: ' + str(len(y_test)))
    print('Model guessed up or down movement correctly ' +str(correct_return/len(pred)) + '%')
    print('mae: ' + str(mae))
    print('mse: ' + str(mse))
    print('R^2: ' + str(r2))

def binance_time():
    """Get server time from Binance in milliseconds"""
    
    server_time = requests.get('https://api.binance.com/api/v3/time').json()['serverTime']
    return server_time

def binance_hash(secret, **kwargs):
    """Encode secret key and message with sha256 and hmac for binance"""
    
    params = urlencode(kwargs)
    hashedsig = hmac.new(secret.encode('utf-8'), params.encode('utf-8'), hashlib.sha256).hexdigest()
    return hashedsig

def binance_params(secret=None,**kwargs):
    """If provided a secret key, add to params"""
    
    params = kwargs
    if secret:
        hashedsig = binance_hash(secret, **params)
        params.update({'signature': hashedsig})
    
    return params

def truncate(f, n):
    "truncates a decimal to n places"
    return math.floor(f * 10 ** n) / 10 ** n